# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Clone to Drive so work persists across sessions
%cd /content/drive/MyDrive
# !git clone https://github.com/hardiman-mostow/IBP_Project.git
%cd IBP_Project/

/content/drive/MyDrive
/content/drive/MyDrive/IBP_Project


In [3]:
!ls

3rd-party-licenses.txt	      configs  logs		 src
acc-rob-tradeoff-cifar10.jpg  data     README.md	 temp_requirements.txt
acc-rob-tradeoff-tin.jpg      LICENSE  requirements.txt


# Push Back to Github

In [4]:
# push back to github
!git config --global user.email ""
!git config --global user.name ""

In [5]:
!git status

Refresh index: 100% (96/96), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   configs/callbacks/rich_progress_bar.yaml
	modified:   configs/experiment/eps_ibp_cifar10.yaml
	modified:   configs/experiment/eps_ibp_mnist.yaml
	modified:   configs/experiment/eps_ibp_tinyimagenet.yaml
	modified:   configs/experiment/eval/eps_ibp_mnist.yaml
	modified:   configs/experiment/fast_ibp_cifar10.yaml
	modified:   configs/experiment/fast_ibp_mnist.yaml
	modified:   configs/experiment/fast_ibp_tinyimagenet.yaml
	modified:   configs/model/eps_ibp_mnist_bn.yaml
	modified:   configs/model/fast_ibp_mnist.yaml
	modified:   src/eval.py
	modified:   src/models/base_bounded_module.py
	modified:   src/models/components/dm_models.py
	modified:   src/models/eps_ibp_module.py
	modified:   src/models/fast_ibp_module.py
	

In [6]:
!git add .

In [7]:
!git commit -m "Update code, bugfixes, basic label smoothing implementation"

[main 10b9e11] Update code, bugfixes, basic label smoothing implementation
 19 files changed, 313 insertions(+), 94 deletions(-)
 create mode 100644 temp_requirements.txt


In [10]:
# See what your current remote is
!git remote -v

origin	https://github.com/hardiman-mostow/IBP_Project.git (fetch)
origin	https://github.com/hardiman-mostow/IBP_Project.git (push)


In [12]:
!git push origin main

Enumerating objects: 52, done.
Counting objects: 100% (52/52), done.
Delta compression using up to 12 threads
Compressing objects: 100% (30/30), done.
Writing objects: 100% (30/30), 6.98 KiB | 340.00 KiB/s, done.
Total 30 (delta 21), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (21/21), completed with 17 local objects.
To https://github.com/hardiman-mostow/IBP_Project.git
   f427477..10b9e11  main -> main


# Imports/Pips

In [4]:
# Install compatible versions
!pip install pytorch-lightning torchmetrics

In [5]:
!pip install git+https://github.com/Verified-Intelligence/auto_LiRPA.git

  Cloning https://github.com/Verified-Intelligence/auto_LiRPA.git to /tmp/pip-req-build-vibnrj0b
  Running command git clone --filter=blob:none --quiet https://github.com/Verified-Intelligence/auto_LiRPA.git /tmp/pip-req-build-vibnrj0b
  Resolved https://github.com/Verified-Intelligence/auto_LiRPA.git to commit cf0169ce6bfb4fddd82cfff5c259c162a23ad03c
  Preparing metadata (setup.py) ... done


In [6]:
!grep -v -E "pytorch-lightning|torchmetrics|auto-lirpa|torch" requirements.txt > temp_requirements.txt
!pip install -r temp_requirements.txt

# Run

In [7]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="torchvision.datasets.mnist")

In [8]:
!python src/train.py experiment=fast_ibp_mnist model.kappa=1 #+trainer.enable_progress_bar=false

[2025-12-03 18:08:26,301][src.utils.utils][INFO] - Disabling python warnings! <cfg.extras.ignore_warnings=True>
[2025-12-03 18:08:26,302][src.utils.utils][INFO] - Enforcing tags! <cfg.extras.enforce_tags=True>
[2025-12-03 18:08:26,310][src.utils.utils][INFO] - Printing config tree with Rich! <cfg.extras.print_config=True>
CONFIG
├── datamodule
│   └── _target_: src.datamodules.mnist_datamodule.MNISTDataModule              
│       data_dir: /content/drive/MyDrive/IBP_Project/data/                      
│       batch_size: 128                                                         
│       train_val_split:                                                        
│       - 55000                                                                 
│       - 5000                                                                  
│       num_workers: 2                                                          
│       pin_memory: false                                                       
│     

In [11]:
!python src/eval.py experiment=eval/eps_ibp_mnist model.kappa=0

[2025-12-03 01:32:32,339][src.utils.utils][INFO] - Disabling python warnings! <cfg.extras.ignore_warnings=True>
[2025-12-03 01:32:32,340][src.utils.utils][INFO] - Enforcing tags! <cfg.extras.enforce_tags=True>
[2025-12-03 01:32:32,347][src.utils.utils][INFO] - Printing config tree with Rich! <cfg.extras.print_config=True>
[2025-12-03 01:32:32,347][src.utils.rich_utils][WARNING] - Field 'callbacks' not found in config. Skipping 'callbacks' config printing...
CONFIG
├── datamodule
│   └── _target_: src.datamodules.mnist_datamodule.MNISTDataModule              
│       data_dir: /content/drive/MyDrive/IBP_Project/data/                      
│       batch_size: 125                                                         
│       train_val_split:                                                        
│       - 55000                                                                 
│       - 5000                                                                  
│       num_workers: 0        

In [11]:
!grep -rn "robust_label_smoothing" configs/

configs/experiment/eps_ibp_cifar10.yaml:80:  robust_label_smoothing: 0.0
configs/experiment/eps_ibp_mnist.yaml:75:  robust_label_smoothing: 0.0
configs/experiment/eps_ibp_tinyimagenet.yaml:79:  robust_label_smoothing: 0.0
configs/experiment/fast_ibp_mnist.yaml:67:  robust_label_smoothing: 0.0
configs/experiment/fast_ibp_cifar10.yaml:77:  robust_label_smoothing: 0.0
configs/experiment/fast_ibp_tinyimagenet.yaml:76:  robust_label_smoothing: 0.0
configs/model/eps_ibp_cifar10_bn.yaml:28:robust_label_smoothing: 0.0
configs/model/eps_ibp_mnist_bn.yaml:30:robust_label_smoothing: 0.1
configs/model/eps_ibp_tinyimagenet_bn.yaml:28:robust_label_smoothing: 0.0
configs/model/fast_ibp_cifar10.yaml:28:robust_label_smoothing: 0.0
configs/model/fast_ibp_mnist.yaml:28:robust_label_smoothing: 0.0
configs/model/fast_ibp_tinyimagenet.yaml:28:robust_label_smoothing: 0.0
configs/model/sabr_cifar10.yaml:28:robust_label_smoothing: 0.0
configs/model/sabr_mnist.yaml:28:robust_label_smoothing: 0.0
configs/model/s